In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
from sklearn.neighbors import NearestNeighbors

In [12]:
from scipy.special import softmax
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import xgboost as xgb
from scipy.optimize import LinearConstraint, Bounds, minimize
import warnings
warnings.filterwarnings("ignore", message="delta_grad == 0.0. Check if the approximated function is linear.")

class SchoolManifold():

    def __init__(self, embeddings_df: pd.DataFrame, k = 15, ):
        self.embeddings_df = embeddings_df
        self.embeddings = self.embeddings_df.drop(labels = ["unitid"], axis =1)

        self.k = k

    
    def _compute_neighbors(self, k: int):
        knn = NearestNeighbors(n_neighbors = k, algorithm="auto")
        knn.fit(self.embeddings.values)
        dists, indices = knn.kneighbors(self.embeddings.values)
        self.neighbor_dists = dists
        self.neighbor_indices = indices


    def preprocess_for_target(self, target_df: pd.DataFrame, target_col_ind: int = None, target_col_lab = None, total_column_lab = None):
        if (target_col_ind is not None):
            target_col_lab = target_df.columns[target_col_ind]

        if (target_col_lab is None):
            raise ValueError("Either target_col_ind or target_col_lab must not be None")

        self.total_column_lab = total_column_lab
        merged = pd.merge(self.embeddings_df, target_df, how= "inner", on = "unitid")
        
        self.embeddings = merged[self.embeddings.columns]
        self._compute_neighbors(self.k)

        self.target_df = merged[target_df.columns]
        self.features = self.target_df.drop(labels = ["unitid", target_col_lab], axis = 1)
        orig_columns = self.features.columns
        self.target = self.target_df[target_col_lab]
        self.schools = self.target_df["unitid"]

        inds_of_target_in_original = merged.index.values
        neighbor_inds_of_target_in_original = self.neighbor_indices[inds_of_target_in_original]
        neighbor_dists_of_target_col = self.neighbor_dists[inds_of_target_in_original]

        vals_of_target_col = merged[target_col_lab].values
        neighbor_vals_of_target_col =  vals_of_target_col[neighbor_inds_of_target_in_original]
        
        features_of_target_col = merged[orig_columns].values
        neighbor_features_of_target_col = features_of_target_col[neighbor_inds_of_target_in_original]

        mask = neighbor_vals_of_target_col > 1.1 * vals_of_target_col.reshape(-1, 1)
        neighbor_weights_of_target_col = np.where(mask, neighbor_dists_of_target_col, np.PINF)
        neighbor_weights_of_target_col = softmax(-neighbor_weights_of_target_col, axis = 1)

        self.mask = mask
        self.v_star = (neighbor_vals_of_target_col * neighbor_weights_of_target_col).sum(axis = 1)
        self.weights =  neighbor_weights_of_target_col
        self.neighbor_features = neighbor_features_of_target_col
        self.neighbor_target = neighbor_vals_of_target_col
        
        if (total_column_lab is not None):
            self.total_column = self.features[total_column_lab].values.reshape(-1,1)
            self.total_column_index = list(self.features.columns).index(total_column_lab)
            self.neighbor_features[:, :, self.total_column_index] = self.total_column
        
        self.vstars = np.where(np.isnan(self.v_star), self.target.values, self.v_star)
    

    def compute_deltas(self, reg, mode = "projection", eps = 0.1):
        self.reg = reg
        self.adjusted_features = np.sum(self.neighbor_features * self.weights[:, :, None], axis = 1)
        self.adjusted_features = np.where(np.isnan(self.adjusted_features), self.features.values, self.adjusted_features)

        if mode == "projection":
            # maintaining total spending constraint
            if (self.total_column_lab):
                # all_but_total_column = [x for x in range(self.features.shape[-1]) if x != self.total_column_index]
                # non_total_adjusted_features = self.adjusted_features[:, all_but_total_column]
                # non_total_features = self.features.values[:, all_but_total_column]
                # non_total_adjusted_features = non_total_adjusted_features / np.sum(non_total_adjusted_features, axis = 1).reshape(-1, 1)
                # non_total_adjusted_features = non_total_adjusted_features * np.sum(non_total_features, axis = 1).reshape(-1, 1)
                # self.adjusted_features[:, all_but_total_column] =  non_total_adjusted_features
                
                self.adjusted_features_df = pd.DataFrame(self.adjusted_features, columns = self.features.columns)
                self.deltas = self.adjusted_features - self.features.values
            
            if type(reg) == xgb.core.Booster:
                self.vhats = reg.predict(xgb.DMatrix(self.adjusted_features_df))
        
        if mode == "optimize":
            def objective_xgb(x, reg, df_cols):
                x_df = pd.DataFrame(x.reshape(1, -1), columns=list(df_cols))
                x = xgb.DMatrix(x_df)
                v = reg.predict(x)
                return -v
            
            def objective(x, reg, df_cols):
                v = reg.predict(pd.DataFrame(x.reshape(1,-1), columns=list(df_cols)))
                return -v
            d = self.adjusted_features.shape[-1]
            positive_constraint_A = np.eye(d)
            positive_constraint = LinearConstraint(positive_constraint_A, lb = 0, ub = np.inf)
            
            distance_constraint_A = np.eye(d)
            self.vhats = []
            for school, x0 in enumerate(self.features.values):
                if (np.allclose(x0, self.adjusted_features[school])):
                    self.vhats.append(self.target[school])
                    continue
                constraints = []

                vstar = self.vstars[school]

                if (self.total_column_lab is not None):
                    ub = np.zeros(d)
                    ub[self.total_column_index] = x0[self.total_column_index]
                    other_cols = [x for x in range(d) if x != self.total_column_index]
                    ub[other_cols] = x0[other_cols] + eps * x0[other_cols]
                    
                    bounds = Bounds(lb = np.zeros(d), ub = ub, keep_feasible=True)
                else:
                    bounds = Bounds(lb = np.zeros(d), ub = x0.flatten() + eps*x0.flatten(), keep_feasible=True)
                
                if type(reg) == xgb.core.Booster:
                    res = minimize(objective_xgb, x0= x0.flatten(), args = (reg, self.features.columns), method="Powell", bounds = bounds)
                else:
                    res = minimize(objective, x0= x0.flatten(), args = (reg, self.features.columns), method="Powell", bounds = bounds)

                self.vhats.append(-res.fun)
                self.adjusted_features[school] = res.x
                print(f"vhat: {-res.fun}, vstar: {vstar}, orig: {self.target[school]}")
                
                
        outcome_results = np.zeros((len(self.vhats), 3))
        outcome_results[:, 0] = self.target[:len(self.vhats)]
        outcome_results[:, 1] = self.vhats[:len(self.vhats)]
        outcome_results[:, 2] = self.vstars[:len(self.vhats)]
        
        outcome_results_df = pd.DataFrame(outcome_results, columns = ["original", "vhat", "vstar"])
        self.adjusted_features_df = pd.DataFrame(self.adjusted_features, columns = self.features.columns)

        self.results = pd.concat((outcome_results_df, self.adjusted_features_df), axis = 1)
        self.deltas = self.adjusted_features - self.features.values

        return self.results

        

### Academic Scores

In [15]:
ic_mca = pd.read_csv("../datasets/Clusters/IC_CLUSTERS_AND_MCA.csv")
ic_mca = ic_mca.loc[:, ic_mca.columns != "ic_cluster"]

academic_scores = pd.read_csv("../datasets/Target Features/academic_scores_per_dollar_scaled.csv", index_col=0)
academic_scores.drop(["log_sat75_per_dollar", "actcm75"], axis= 1, inplace= True)

In [16]:
academic_scores

,unitid,log_act75_per_dollar,Total expenses,expense_per_student_per_dollar,Institutional grants (funded)_per_dollar,Institutional grants (unfunded)_per_dollar,Sales and services of educational activities - Total_per_dollar,Sales and services of auxiliary enterprises - Total_per_dollar,Instruction-Total amount_per_dollar,Research-Total amount_per_dollar,Public service-Total amount_per_dollar,Academic support-Total amount_per_dollar,Student service-Total amount_per_dollar,Institutional support-Total amount_per_dollar,Auxiliary enterprises-Total amount_per_dollar,Net grant aid to students-Total amount_per_dollar
0,100937,0.379414,-0.171030,-0.232716,1.206151,0.412490,-0.280020,2.027240,-0.205684,-0.296775,-0.329219,-0.173034,0.601122,-0.015549,1.290770,-0.038351
1,101073,1.551688,-0.226580,-0.217027,-0.567865,-0.996212,-0.280020,0.454750,-0.199023,-0.296775,-0.329219,-0.626781,-0.806552,1.038228,1.944676,-0.194775
2,101189,0.017413,-0.174928,-0.228830,-0.388847,0.025353,-0.280020,-0.654971,0.419387,-0.296775,-0.329219,-0.183660,0.664577,-0.019002,-0.154420,-0.194775
3,101365,0.958293,-0.235120,-0.123286,-0.567865,-0.401752,-0.280020,-1.426835,0.464962,-0.296775,-0.329219,-0.419549,1.714756,-0.305268,-1.476223,-0.194775
4,101435,0.794234,-0.208896,-0.224897,2.082840,0.126760,-0.280020,0.758882,-0.261806,-0.296775,-0.329219,-0.123677,1.694626,-0.252428,0.534649,-0.194775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,494463,0.148389,-0.184421,-0.223204,-0.567865,-1.255182,-0.280020,0.090170,-2.257439,-0.296775,-0.329219,-1.247129,-1.752056,-1.970884,0.118338,-0.194775
1334,494685,1.799204,-0.239801,-0.102113,-0.419833,-1.003053,-0.280020,-1.426835,-0.359637,-0.296775,-0.329219,0.607929,1.680270,0.653051,-1.476223,-0.194775
1335,495031,-0.624960,-0.241074,1.339027,-0.567865,1.845181,-0.280020,-1.426835,-0.168897,-0.296775,0.875965,-1.355850,-0.769684,-1.073563,-1.476223,10.660086
1336,495767,-1.107471,11.743455,-0.261632,-0.400906,-1.239166,0.364676,-0.880177,-1.118511,2.853822,0.355841,-0.520190,-1.498597,-1.520819,-0.765774,-0.194775


In [17]:
manifold = SchoolManifold(ic_mca)

In [18]:
# yeff_no_total = yield_efficiency.drop("Total expenses", axis = 1)
manifold.preprocess_for_target(academic_scores, target_col_lab="log_act75_per_dollar", total_column_lab="Total expenses")

/home/acowlagi/.conda/envs/datathon/lib/python3.9/site-packages/scipy/special/_logsumexp.py:214: RuntimeWarning: invalid value encountered in subtract
  return np.exp(x - logsumexp(x, axis=axis, keepdims=True))


In [20]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
import pickle

xgb_reg = xgb.Booster()
xgb_reg.load_model("../models/act75_model.json")

[14:19:13] WARNING: ../src/learner.cc:888: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.


In [21]:
manifold.compute_deltas(xgb_reg, mode = "optimize")

/tmp/ipykernel_19707/2210537873.py:130: OptimizeWarning: Initial guess is not within the specified bounds
  res = minimize(objective_xgb, x0= x0.flatten(), args = (reg, self.features.columns), method="Powell", bounds = bounds)


vhat: 0.4227732717990875, vstar: 0.5726446993110135, orig: 0.379413994518644


/tmp/ipykernel_19707/2210537873.py:130: OptimizeWarning: Initial guess is not within the specified bounds
  res = minimize(objective_xgb, x0= x0.flatten(), args = (reg, self.features.columns), method="Powell", bounds = bounds)


vhat: 0.15585972368717194, vstar: 0.4125721481358695, orig: 0.0174127467064182
vhat: 0.881549060344696, vstar: 2.2914104032190394, orig: 0.95829270106815


/tmp/ipykernel_19707/2210537873.py:130: OptimizeWarning: Initial guess is not within the specified bounds
  res = minimize(objective_xgb, x0= x0.flatten(), args = (reg, self.features.columns), method="Powell", bounds = bounds)
/tmp/ipykernel_19707/2210537873.py:130: OptimizeWarning: Initial guess is not within the specified bounds
  res = minimize(objective_xgb, x0= x0.flatten(), args = (reg, self.features.columns), method="Powell", bounds = bounds)


...

vhat: -0.7805138230323792, vstar: -0.08327926877141961, orig: -1.0815347314417365


/tmp/ipykernel_19707/2210537873.py:130: OptimizeWarning: Initial guess is not within the specified bounds
  res = minimize(objective_xgb, x0= x0.flatten(), args = (reg, self.features.columns), method="Powell", bounds = bounds)


vhat: 0.31378602981567383, vstar: 0.47858407280958154, orig: 0.1483888040765348


/tmp/ipykernel_19707/2210537873.py:130: OptimizeWarning: Initial guess is not within the specified bounds
  res = minimize(objective_xgb, x0= x0.flatten(), args = (reg, self.features.columns), method="Powell", bounds = bounds)


KeyboardInterrupt: 

### Testing Area

In [484]:
df = yield_efficiency.drop("Total expenses", axis = 1)

In [485]:
df

,unitid,log_yield_per_dollar,Institutional grants (funded)_per_dollar,Institutional grants (unfunded)_per_dollar,Sales and services of educational activities - Total_per_dollar,Sales and services of auxiliary enterprises - Total_per_dollar,Instruction-Total amount_per_dollar,Research-Total amount_per_dollar,Public service-Total amount_per_dollar,Academic support-Total amount_per_dollar,Student service-Total amount_per_dollar,Institutional support-Total amount_per_dollar,Auxiliary enterprises-Total amount_per_dollar,Net grant aid to students-Total amount_per_dollar
0,100937,0.111066,0.120239,0.349113,0.000000,0.327833,0.308731,0.000000,0.000000,0.070772,0.204256,0.197669,0.211686,0.006885
1,101073,2.544717,0.000000,0.066926,0.000000,0.178585,0.309677,0.000000,0.000000,0.043623,0.084801,0.300184,0.261713,0.000000
2,101189,1.001058,0.012133,0.271563,0.000000,0.073259,0.397523,0.000000,0.000000,0.070137,0.209641,0.197334,0.101123,0.000000
3,101365,2.801799,0.000000,0.186007,0.000000,0.000000,0.403997,0.000000,0.000000,0.056023,0.298759,0.169485,0.000000,0.000000
4,101435,0.887857,0.179659,0.291876,0.000000,0.207451,0.300758,0.000000,0.000000,0.073726,0.297051,0.174625,0.153840,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,494463,0.661397,0.000000,0.015050,0.000000,0.143982,0.017276,0.000000,0.000000,0.006505,0.004565,0.007449,0.121990,0.000000
1334,494685,6.594540,0.010033,0.065556,0.000000,0.000000,0.286861,0.000000,0.000000,0.117500,0.295833,0.262713,0.000000,0.000000
1335,495031,5.185335,0.000000,0.636105,0.000000,0.000000,0.313956,0.000000,0.025573,0.000000,0.087929,0.094743,0.000000,0.477798
1336,495767,-4.521554,0.011316,0.018258,0.014545,0.051884,0.179063,0.123366,0.014536,0.050001,0.026074,0.051233,0.054352,0.000000


In [497]:
from sklearn.model_selection import train_test_split
X, y = df.iloc[:, 2:], df.iloc[:, 1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators = 20, random_state=0)

regr.fit(X_train, y_train)

RandomForestRegressor(n_estimators=20, random_state=0)

In [498]:
from sklearn.metrics import r2_score, mean_squared_error

print(f"R^2: {r2_score(y_test, regr.predict(X_test))}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, regr.predict(X_test)))}")

R^2: 0.5711633088312353
RMSE: 1.4945370933080349


In [ ]:
test = np.array([[1,2, 3], [2,3, 4], [3,4, 5], [4, 5, 6]])
mask = test > np.array([2, 3, 4, 5]).reshape(-1, 1)

altered_test = np.where(mask, test, np.PINF)
softmax(-altered_test, axis = 1)

array([[-inf, -inf,  -3.],
       [-inf, -inf,  -4.],
       [-inf, -inf,  -5.],
       [-inf, -inf,  -6.]])

In [ ]:
df = pd.DataFrame(np.random.randn(50, 1), columns=list('A'))
df["E"] = 2*np.arange(0, 50)
df2 = pd.DataFrame(np.random.randn(100, 1), columns=list('B'))
df2 = pd.DataFrame(np.random.randn(100, 1), columns=list('B'))

In [ ]:
df3 = pd.merge(df2.reset_index(), df,  how="right", on = "E").set_index('index')
df3

In [5]:
neighbor_values = np.arange(24).reshape(4, 3, 2)
weights = np.array([[0.3, 0.3, 0.4], [0.9, 0.1, 0], [0.6, 0.2, 0.2], [0, 0, 1]])

In [6]:
np.sum(neighbor_values * weights[:, :, None], axis = 1)

array([[ 2.2,  3.2],
       [ 6.2,  7.2],
       [13.2, 14.2],
       [22. , 23. ]])

In [8]:
neighbor_values[:, :, 1] = np.arange(4).reshape(-1,1)

In [9]:
neighbor_values

array([[[ 0,  0],
        [ 2,  0],
        [ 4,  0]],

       [[ 6,  1],
        [ 8,  1],
        [10,  1]],

       [[12,  2],
        [14,  2],
        [16,  2]],

       [[18,  3],
        [20,  3],
        [22,  3]]])

In [ ]:
# if self.total_column_lab is not None:
#     lb[self.total_column_index] = x0[self.total_column_index]
#     ub[self.total_column_index] = x0[self.total_column_index]
#     all_but_total = [x for x in range(d) if x != self.total_column_index]
#     lb[all_but_total] = x0[all_but_total] - eps * x0[all_but_total]
#     ub[all_but_total] = x0[all_but_total] + eps * x0[all_but_total]
# else:
#     lb = x0 - eps * x0
#     ub = x0 + eps * x0